In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
pip install https://h2o-release.s3.amazonaws.com/h2o/rel-zeno/3/Python/h2o-3.30.1.3-py2.py3-none-any.whl

In [ ]:
import h2o
import numpy as np
import os
import pandas as pd
import time

from h2o.automl import H2OAutoML
from PIL import Image

In [ ]:
%matplotlib inline

In [ ]:
def image_crop(image):
    tar_h, tar_w = 7057, 943
    im_h, im_w = image.shape

    h_gap = im_h - tar_h
    if h_gap % 2 == 0:
        a = h_gap // 2
        b = im_h - h_gap // 2
    else:
        a = int(np.floor(h_gap / 2))
        b = im_h - int(np.ceil(h_gap / 2))

    w_gap = im_w - tar_w
    if w_gap % 2 == 0:
        c = w_gap // 2
        d = im_w - w_gap // 2
    else:
        c = int(np.floor(w_gap / 2))
        d = im_w - int(np.ceil(w_gap / 2))

    box = image[a:b, c:d]
    return box

In [ ]:
def instanceSelect(selection, m, two=False, ten=False):
    key = [
        "20122017",  # 0
        "17012018",  # 1
        "23012018",  # 2
        "20022018",  # 3
        "05022018",  # 4
        "06032018",  # 5
        "13032018",  # 6
        "28032018",  # 7
        "20032018",  # 8
    ]

    all_data = []

    im1 = Image.open(f"{main_path}/UAS Data/MicaSense/NTR{key[selection]}.tif")

    im1array = np.array(im1)
    im1array = image_crop(im1array)
    imNTR = np.reshape(im1array, (7057 * 943, 1))

    NTR = pd.DataFrame(imNTR,columns={"NTR"})
    all_data.append(NTR)

    im2 = Image.open(f"{main_path}/UAS Data/MicaSense/NDVI{key[selection]}.tif")

    im2array = np.array(im2)
    im2array = image_crop(im2array)
    imNDVI = np.reshape(im2array, (7057 * 943, 1))

    NDVI = pd.DataFrame(imNDVI,columns={"NDVI"})
    all_data.append(NDVI)

    if two:
        im3 = Image.open(f"{main_path}/raw_maps/map_2cm_mode_{m}_{key[selection]}.tif")

        im3array = np.array(im3)
        im3array = image_crop(im3array)
        imtwo_cm_pred = np.reshape(im3array, (7057 * 943, 1))

        two_cm_pred = pd.DataFrame(imtwo_cm_pred,columns={"2 cm"})
        all_data.append(two_cm_pred)

    if ten:
        im4 = Image.open(f"{main_path}/raw_maps/map_10cm_mode_{m}_{key[selection]}.tif")

        im4array = np.array(im4)
        im4array = image_crop(im4array)
        imten_cm_pred = np.reshape(im4array, (7057 * 943, 1))

        ten_cm_pred = pd.DataFrame(imten_cm_pred,columns={"10 cm"})
        all_data.append(ten_cm_pred)

    date = key[select]

    return all_data, date

In [ ]:
def featureSelect(depth, mode=0, check=False):
    """
    enter only 2/10/50 for depth
    """
    if depth == 2 or depth == 10:
        alt = "L1"
    elif depth == 50:
        alt = "L3"

    ntr = ["NTR"]
    ndvi = ["NDVI"]
    psd = [f"Clay_{alt}", f"Sand_{alt}", f"Silt_{alt}"]
    bd = [f"BD_{alt}_2018"]
    om = [f"OM_{alt}_2018"]
    thetaFC = [f"θFC_{alt}"]
    thetaPWP = [f"θPWP_{alt}"]
    ksat = [f"Ksat_{alt}"]
    porosity = [f"Porosity_{alt}_2018"]

    feature_combo = [
        ntr,  # 0
        ntr + ndvi,  # 1
        ntr + psd,  # 2
        ntr + ndvi + psd,  # 3
        ntr + ndvi + psd + bd,  # 4
        ntr + ndvi + psd + bd + om,  # 5
        ntr + ndvi + psd + bd + thetaFC,  # 6
        ntr + ndvi + psd + bd + thetaFC + thetaPWP,  # 7
        ntr + ndvi + thetaFC + thetaPWP,  # 8
        ntr + ndvi + psd + bd + om + thetaFC + thetaPWP,  # 9
        ntr + ndvi + psd + bd + om + thetaFC + thetaPWP + ksat,  # 10
        ntr + ndvi + psd + om + porosity,  # 11
    ]

    if check:
        print([f"{depth} cm"] + feature_combo[mode])

    return feature_combo[mode]

In [ ]:
def featureImageSelect(depth, mode=1, check=False):
    """
    enter only 2/10/50 for depth
    """
    if depth == 2 or depth == 10:
        alt = "L1"
        len = "6"
    elif depth == 50:
        alt = "L3"
        len = "18"

    ntr = []
    ndvi = []
    psd = [f"Clay_{len}inch.tif", f"Sand_{len}inch.tif", f"Silt_{len}inch.tif"]
    bd = [f"BulkDensity_2018_{len}inch.tif"]
    om = [f"OrganicMatter_2018_{len}inch.tif"]
    thetaFC = [f"Theta_FC_{alt}.tif"]
    thetaPWP = [f"Theta_PWP_{alt}.tif"]
    ksat = [f"Ksat_{len}inch.tif"]
    porosity = [f"Porosity_2018_{len}inch.tif"]

    feature_combo = [
        ntr,  # 0
        ntr + ndvi,  # 1
        ntr + psd,  # 2
        psd,  # 3
        psd + bd,  # 4
        psd + bd + om,  # 5
        psd + bd + thetaFC,  # 6
        psd + bd + thetaFC + thetaPWP,  # 7
        thetaFC + thetaPWP,  # 8
        psd + bd + om + thetaFC + thetaPWP,  # 9
        psd + bd + om + thetaFC + thetaPWP + ksat,  # 10
        psd + om + porosity,  # 11
    ]

    if check:
        print(feature_combo[mode])

    return feature_combo[mode]

In [ ]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpi70ddkzh
  JVM stdout: /tmp/tmpi70ddkzh/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpi70ddkzh/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,1 month and 30 days
H2O_cluster_name:,H2O_from_python_unknownUser_mbpf9i
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


# Select features

In [ ]:
d = 50
m = 1

istwo = True
isten = True

In [ ]:
pop = featureImageSelect(d, m)
blue = featureSelect(d, m)

In [ ]:
if isten:
    blue.insert(2, "10 cm")
    est = "10cm"
    sub_path = "run_4_with_10cm"
if istwo:
    blue.insert(2, "2 cm")
    est = "2cm"
    sub_path = "run_3_with_2cm"
if istwo and isten:
    est = "BOTH"
    sub_path = "run_5_with_BOTH"

In [ ]:
blue

['NTR', 'NDVI', '2 cm', '10 cm']

In [ ]:
main_path = "/content/gdrive/My Drive/Soil moisture project"

box = os.listdir(f"{main_path}/models/{sub_path}/data_{d}_{m}")
saved_model = h2o.load_model(f"{main_path}/models/{sub_path}/data_{d}_{m}/{box[0]}")

In [ ]:
print(saved_model)

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid__3_AutoML_20201103_012430_model_1


Status of Neuron Layers: predicting 50 cm, regression, gaussian distribution, Quadratic loss, 504,001 weights/biases, 5.8 MB, 12,450 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,4,Input,5,,,,,,,,,
1,,2,500,RectifierDropout,0,0,0,0.0276842,0.0142769,0,0.00542389,0.0922142,0.477728,0.0172848
2,,3,500,RectifierDropout,0,0,0,0.068039,0.104735,0,-0.00579083,0.046482,0.987942,0.00436095
3,,4,500,RectifierDropout,0,0,0,0.156466,0.238808,0,-0.00145756,0.046599,0.827846,0.175401
4,,5,1,Linear,,0,0,0.000852709,0.000482073,0,-9.45659e-05,0.0514715,0.000406999,1.09713e-154




ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.00023082694324657061
RMSE: 0.01519298993768411
MAE: 0.011989287152496076
RMSLE: 0.012932014886888981
Mean Residual Deviance: 0.00023082694324657061

ModelMetricsRegression: deeplearning
** Reported on validation data. **

MSE: 0.0009168563639507699
RMSE: 0.030279636126459147
MAE: 0.025770407380918445
RMSLE: 0.02510958404777791
Mean Residual Deviance: 0.0009168563639507699

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.0003062900325424738
RMSE: 0.017501143749551737
MAE: 0.014574928221770866
RMSLE: 0.015255929672093254
Mean Residual Deviance: 0.0003062900325424738

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.014574928,0.005710265,0.022045324,0.012014715,0.017893514,0.007054837,0.013866252
1,mean_residual_deviance,3.0629002E-4,2.093314E-4,6.031476E-4,1.7641553E-4,3.9247415E-4,5.5585104E-5,3.0382778E-4
2,mse,3.0629002E-4,2.093314E-4,6.031476E-4,1.7641553E-4,3.9247415E-4,5.5585104E-5,3.0382778E-4
3,r2,-5.974781,15.310306,0.77140933,0.8060959,-33.362267,0.97239065,0.9384649
4,residual_deviance,3.0629002E-4,2.093314E-4,6.031476E-4,1.7641553E-4,3.9247415E-4,5.5585104E-5,3.0382778E-4
5,rmse,0.016507674,0.0064987224,0.024559064,0.013282151,0.01981096,0.0074555418,0.017430656
6,rmsle,0.0143524725,0.005782508,0.021898054,0.011657032,0.016747708,0.006395993,0.015063578



Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,training_r2,validation_rmse,validation_deviance,validation_mae,validation_r2
0,,2020-11-03 01:34:00,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-11-03 01:34:02,1 min 52.247 sec,75 obs/sec,10.0,1,150.0,0.093840,0.008806,0.091093,-1.969980,0.098107,0.009625,0.095633,-4.600858
2,,2020-11-03 01:34:08,1 min 57.245 sec,216 obs/sec,100.0,10,1500.0,0.016192,0.000262,0.013055,0.911572,0.028342,0.000803,0.022677,0.532572
3,,2020-11-03 01:34:13,2 min 2.402 sec,348 obs/sec,280.0,28,4200.0,0.015080,0.000227,0.010951,0.923306,0.024894,0.000620,0.022349,0.639370
4,,2020-11-03 01:34:18,2 min 7.584 sec,418 obs/sec,480.0,48,7200.0,0.018269,0.000334,0.014964,0.887439,0.033553,0.001126,0.029180,0.344896
5,,2020-11-03 01:34:23,2 min 12.650 sec,459 obs/sec,680.0,68,10200.0,0.017812,0.000317,0.012148,0.892991,0.024198,0.000586,0.020281,0.659262
6,,2020-11-03 01:34:27,2 min 16.396 sec,480 obs/sec,830.0,83,12450.0,0.015193,0.000231,0.011989,0.922149,0.030280,0.000917,0.025770,0.466472



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,10 cm,1.000000,1.000000,0.313931
1,NDVI,0.762305,0.762305,0.239311
2,NTR,0.757643,0.757643,0.237847
3,2 cm,0.665469,0.665469,0.208911


In [ ]:
start_time = time.time()

In [ ]:
for select in range(9):
    bag, date = instanceSelect(select, m, istwo, isten)

    for feature in pop:
        im = Image.open(f"{main_path}/Soil Parameters/{feature}")

        imarray = np.array(im)
        imarray = image_crop(imarray)
        flat = np.reshape(imarray, (7057 * 943, 1))

        bag.append(pd.DataFrame(flat,columns={feature}))

    data = pd.concat(bag, axis=1)

    if select == 0:
        print(data.columns)
        print(blue)
        print("\n")
    
    data.columns = blue

    test = h2o.H2OFrame(data)
    preds = saved_model.predict(test)

    box = preds.as_data_frame().values
    map = np.reshape(box, (7057, 943))
    im = Image.fromarray(map)
    im.save(f"{main_path}/raw_maps/map_{d}cm_mode_{m}_with_{est}_estimation_{date}.tif")

    interval_time = time.time()

    print(f"Pass {select + 1} of 9 completed")

    hours, rem = divmod(interval_time - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:02.0f}".format(int(hours), int(minutes) ,seconds))
    print("\n\n---------------------------------------------------------------------------\n\n")

In [ ]:
print("\n-----------------------------------------------End of process-----------------------------------------------")